In [2]:
from konlpy.tag import Okt
from gensim.models.fasttext import FastText, load_facebook_vectors, load_facebook_model
import os
import pandas as pd
import re
from gensim.test.utils import datapath

def preprocess_korean_text(text, stop_words=[]):
    okt = Okt()
    
    # 형태소 분석을 통한 토큰화
    words = okt.morphs(text, norm=True) # 정규화 처리
    
    # 불용어 제거
    words = [word for word in words if word not in stop_words]
    
    return words

def process_csv_file(csv_path, stop_words=[]):
    new_data = []
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    if 'reviews' in df.columns:
        for text in df['reviews']:
            # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
            text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
            
            # 전처리된 문장을 추가 데이터에 포함
            tokens = preprocess_korean_text(text, stop_words)
            new_data.append(tokens)
    
    return new_data


# 기존 pre-trained 모델 경로
pretrained_model_path = 'wiki.ko.bin'

# 새로운 데이터 파일 경로
csv_file_path = '../ReviewData/reviews_only.csv'

# 불용어 리스트 (필요에 따라 추가)
stop_words = ['의','가', '에','들','는','잘','걍','과','도','를','으로','한','하다','!','?','<','>','(',')','[',']','|','#','.', '이','은','는','을','에','에서','로']

# 모델 로드
# model = load_facebook_vectors(pretrained_model_path)
model = load_facebook_model(pretrained_model_path)

# CSV 파일 처리 및 전처리
new_data = process_csv_file(csv_file_path, stop_words)

# 모델 업데이트
model.build_vocab(corpus_iterable=new_data, update=True)
model.train(corpus_iterable=new_data, total_examples=len(new_data), epochs=model.epochs)

# 업데이트된 모델 저장
updated_model_path = 'updated_model.bin'
model.save(updated_model_path)

# 저장된 모델 경로 출력
print(f"모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: {os.path.abspath(updated_model_path)}")


KeyboardInterrupt



In [5]:
from gensim.models.fasttext import FastText

# 저장한 모델 불러오기
model = FastText.load('./updated_model.bin')

def find_similar_words(word, topn=10):
    try:
        # 입력한 단어와 유사한 단어들을 찾기
        similar_words = model.wv.most_similar(word, topn=topn)

        # 결과 출력
        print(f"입력한 단어 '{word}'와(과) 유사한 단어들:")
        for similar_word, similarity in similar_words:
            print(f"{similar_word}: {similarity}")

    except KeyError:
        print(f"입력한 단어 '{word}'는 모델의 어휘에 없습니다.")

# 사용자로부터 단어 입력 받기
user_input = input("단어를 입력하세요: ")

# 유사한 단어 찾기 및 출력
find_similar_words(user_input)

입력한 단어 '치킨'와(과) 유사한 단어들:
양념치킨: 0.8387115001678467
치킨과: 0.8261151909828186
치킨윙: 0.8219879865646362
치킨에: 0.819800078868866
치킨이: 0.809363842010498
치킨은: 0.7859618663787842
치킨을: 0.785783588886261
치킨의: 0.777712881565094
먼치킨: 0.7542405724525452
치킨집: 0.7345045804977417


In [4]:
# 조금 변형해본 코드
from konlpy.tag import Okt
from gensim.models.fasttext import FastText, load_facebook_vectors, load_facebook_model
import os
import pandas as pd
import re
from gensim.test.utils import datapath

def preprocess_korean_text(text, stop_words=[], pos_tags=['Noun']):
    okt = Okt()
    
    # 형태소 분석을 통한 토큰화 및 품사 태깅
    morphs = okt.pos(text, norm=True, stem=True)
    
    # 선택한 품사의 단어만 추출
    words = [word for word, pos in morphs if pos in pos_tags]
    
    # 불용어 제거
    words = [word for word in words if word not in stop_words]
    
    return words

def process_csv_file(csv_path, stop_words=[]):
    new_data = []
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    if 'reviews' in df.columns:
        for text in df['reviews']:
            # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
            text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
            
            # 전처리된 문장을 추가 데이터에 포함
            tokens = preprocess_korean_text(text, stop_words)
            new_data.append(tokens)
    
    return new_data


# 기존 pre-trained 모델 경로
pretrained_model_path = 'wiki.ko.bin'

# 새로운 데이터 파일 경로
csv_file_path = '../ReviewData/reviews_only.csv'

# 불용어 리스트 (필요에 따라 추가)
stop_words = ['의','가', '에','들','는','잘','걍','과','도','를','으로','한','하다','!','?','<','>','(',')','[',']','|','#','.', '이','은','는','을','에','에서','로']

# 모델 로드
# model = load_facebook_vectors(pretrained_model_path)
model = load_facebook_model(pretrained_model_path)

# CSV 파일 처리 및 전처리
new_data = process_csv_file(csv_file_path, stop_words)

# 모델 업데이트
model.build_vocab(corpus_iterable=new_data, update=True)
model.train(corpus_iterable=new_data, total_examples=len(new_data), epochs=model.epochs)

# 업데이트된 모델 저장
updated_model_path = 'updated_model.bin'
model.save(updated_model_path)

# 저장된 모델 경로 출력
print(f"모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: {os.path.abspath(updated_model_path)}")

모델이 성공적으로 업데이트되었습니다. 업데이트된 모델 경로: E:\NLP_PROJECT\NLP_Final_Project\Preprocess\updated_model.bin


In [1]:
from gensim.models.fasttext import FastText

# 저장한 모델 불러오기
model = FastText.load('./updated_model.bin')

def find_similar_words(word, topn=10):
    try:
        # 입력한 단어와 유사한 단어들을 찾기
        similar_words = model.wv.most_similar(word, topn=topn)

        # 결과 출력
        print(f"입력한 단어 '{word}'와(과) 유사한 단어들:")
        for similar_word, similarity in similar_words:
            print(f"{similar_word}: {similarity}")

    except KeyError:
        print(f"입력한 단어 '{word}'는 모델의 어휘에 없습니다.")

# 사용자로부터 단어 입력 받기
user_input = input("단어를 입력하세요: ")

# 유사한 단어 찾기 및 출력
find_similar_words(user_input)

KeyboardInterrupt: 

In [9]:
from konlpy.tag import Okt
from gensim.models.fasttext import FastText
import os
import pandas as pd
import re
from gensim.utils import simple_preprocess

def preprocess_korean_text(text, stop_words=[]):
    okt = Okt()
    
    # 형태소 분석을 통한 토큰화
    morphs = okt.morphs(text, norm=True, stem=True)
    
    # 불용어 제거
    words = [word for word in morphs if word not in stop_words]
    
    return words

def process_csv_file(csv_path, stop_words=[]):
    new_data = []
    
    df = pd.read_csv(csv_path, encoding='utf-8')
    if 'reviews' in df.columns:
        for text in df['reviews']:
            # 추가적인 전처리 작업 (예: 특수 문자 제거, 숫자 제거 등)
            text = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z\s]', '', text)
            
            # 전처리된 문장을 추가 데이터에 포함
            tokens = preprocess_korean_text(text, stop_words)
            new_data.append(tokens)
    
    return new_data

# 새로운 데이터 파일 경로
csv_file_path = '../ReviewData/reviews_only.csv'

# 불용어 리스트 (필요에 따라 추가)
stop_words = ['의','가', '에','들','는','잘','걍','과','도','를','으로','한','하다','!','?','<','>','(',')','[',']','|','#','.', '이','은','는','을','에','에서','로']

# 모델 초기화
model = FastText(vector_size=100, window=5, min_count=5, workers=4, sg=1)

# CSV 파일 처리 및 전처리
new_data = process_csv_file(csv_file_path, stop_words)

# 모델 학습
model.build_vocab(corpus_file=simple_preprocess(csv_file_path)[0])  # 이 부분 수정
model.train(corpus_iterable=new_data, total_examples=len(new_data), epochs=10)

# 모델 저장
model_path = 'new_model.bin'
model.save(model_path)

# 저장된 모델 경로 출력
print(f"모델이 성공적으로 생성되었습니다. 모델 경로: {os.path.abspath(model_path)}")


TypeError: Parameter corpus_file must be a valid path to a file, got 'reviewdata' instead

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from konlpy.tag import Okt

# CSV 파일 경로
csv_file_path = '../ReviewData/reviews_converted.csv'

# CSV 파일 읽기
df = pd.read_csv(csv_file_path, encoding='utf-8')

# 형태소 분석을 위한 객체 생성
okt = Okt()

# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(tokenizer=okt.morphs, max_features=1000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df['reviews'])

# K-means 군집화
num_clusters = 5  # 군집 수는 적절하게 조절하세요.
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)

# 각 리뷰의 군집 할당 결과
clusters = kmeans.labels_

# 결과를 DataFrame에 추가
df['cluster'] = clusters

# 군집 결과 출력
for cluster_id in range(num_clusters):
    cluster_data = df[df['cluster'] == cluster_id]['reviews'].head(5).tolist()
    print(f"Cluster {cluster_id + 1}:\n", cluster_data)



E:\Anaconda3_envs\nlp_venv\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
E:\Anaconda3_envs\nlp_venv\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Cluster 1:
 ['너어어어어어무 맛있게 잘 먹었어요ㅋㅋ', '잘먹었어요', '맛있게 잘 먹었어요^^', '정말 맛있게 잘 먹었어요ㅋㅋ', '맛있게 잘먹었어요 ㅎ !!!']
Cluster 2:
 ['맛있게 잘먹었습니다', '맛있게 잘 먹었습니다.', '맛있게 잘 먹었습니다', '맛있게 잘먹었습니다', '맛있게 잘 먹었습니다']
Cluster 3:
 ['잡채밥은 첨시켰는데 맛있어요!', '너무 맛있어요！！！！！！！！', '맛있어요. 배달 도 빨라요', '맛있어요맛있어요맛있어요', '십분만에 왔어요 맛있어요']
Cluster 4:
 ['배달 초스피드 개맛있음', '배달도 빠르고 가격도 착하고 맛도 굿', '배달도 빠르고 양이 엄청 많아여', '배달 빠르고 맛있어요', '항상 배달도 빠르고 맛있어요 잘먹었습니다']
Cluster 5:
 ['오랜만에  완두콩,옥수수 들어있는 짜장면을 먹어보네요!배달도 무척이나 빠르고짬뽕국물도 서비스로 오니 야식 시킬때 정말 딱입니다어 근데  제입맛엔  조금 짰어요', '맛과 양까지 다 최고예요', '아니 제가 도착하면 문자해달라고 체크해놧는데한시간 넘게 지나서도 안오길레 문자가 혹시나 해서문밖에 보니깐 음식 와있네요^^이미 음식은 추워서 다 식엇고 차가운상태네요문자하나 써주는게 뭐가 어렵다고 참나다신 안시켜먹을듯 하네요', '탕수육은 맛있었는데 짬뽕, 짜장은 제 입맛에는 좀짜게 느껴지네요.담엔 싱겁게 요청드려야겠네요', '1인분배달되는곳중에 제일 맛있어서 여기서만 시키려구요']


In [ ]:
from gensim.models import fasttext
fasttext.sav